## 일급 객체의 조건
- 런타임에 생성가능
- 데이터 구조체의 변수나 요소에 할당 가능
- 함수의 인수로 전달 가능
- 함수의 결과로 반환 가능
- 예시 : 정수, 문자열, 딕셔너리 등

# 5.1 함수를 객체처럼 다루기
- 함수 역시 function class 의 객체이다

In [1]:
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n-1)

In [2]:
factorial(10)

3628800

In [3]:
factorial.__doc__

'returns n!'

In [4]:
type(factorial)

function

In [5]:
help(factorial)

Help on function factorial in module __main__:

factorial(n)
    returns n!



In [6]:
fact = factorial
fact

<function __main__.factorial(n)>

In [7]:
fact(5)

120

In [8]:
map(factorial, range(11))

In [9]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

# 5.2 고위 함수
- higher-order function
- 함수를 인수로 받거나, 함수를 결과로 반환하는 함수
- map, sorted

In [10]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspbrry', 'banana']
sorted(fruits, key=len) #len 함수를 받음

['fig', 'apple', 'cherry', 'banana', 'raspbrry', 'strawberry']

In [11]:
def reverse(word):
    return word[::-1]
reverse('testing')

'gnitset'

In [12]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspbrry', 'strawberry', 'cherry']

In [13]:
# 익명함수
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspbrry', 'strawberry', 'cherry']

# 5.4 callable objects
- 사용자 정의 함수 : def 로 시작
- 내장 함수 : len or time.strftime 등 c로 구현된 함수
- 내장 메서드 : dict.get() 등 c로 구현된 함수 
- 메서드 : 클래스 본체에 정의된 함수
- 클래스 : 호출시, 자신의 __new__() 메서드를 실행해 객체 생성, __init__() 초기화 후 객체 리턴
- 파이썬에는 new 연산자가 없어 클래스 호출은 함수 호출과 동일
- 클래스 객체 : __call__() 메서드를 구현하면 함수로 호출 가능 
- 제너레이터 함수 : yield 키워드를 사용하는 함수나 메서드, 제너레이터 객체를 반환

In [14]:
abs, str, 13

(<function abs(x, /)>, str, 13)

In [15]:
# callable 판단
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

# 5.5 사용자 정의 콜러블형
- __call__() 메서드를 구현하여 함수처럼 동작

In [16]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('empty BingoCage')

    def __call__(self):
        return self.pick()

In [19]:
bingo = BingoCage(range(30))
bingo.pick(), bingo(), callable(bingo)

(10, 26, True)

# 5.6 함수 인트로스펙션 
- 함수 객체는 다양한 속성을 가지고 있다

In [20]:
 dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [21]:
# __dict__ 속성 : 객체에 할당된 사용자 속성을 보관

class C: pass
obj = C()
def func(): pass

sorted(set(dir(func)) - set(dir(obj))) #function 에만 존재하는 속성

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

# 5.7 위치 매개변수에서 키워드 전용 매개변수까지 

In [22]:
def tag(name, *content, cls=None, **attrs):
    """하나 이상의 HTML 태그를 생성 """
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''

    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [23]:
tag('p', 'hello') # 첫 번째 이후 인수들은 모두 *content 매개변수에 튜플로 전달된다.

'<p>hello</p>'

In [24]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [25]:
my_tag = {'name':'img', 'title':'Sunset Boulevard', 'src':'sunset.jpg', 'cls':'framed'}
tag(**my_tag) # 명명된 매개변수 외 나머지는 **attrs에 전달

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [27]:
def f(a=1, *, b=2):
    return a,b
f(1,b=2)

(1, 2)

# 5.8 매개변수 정보 읽기
- 함수 객체 안의 __defaults__ 속성에는 위치 인수와 키워드 인수의 기본값을 가진 튜플이 들어 있다. 
- 키워드 전용 인수의 기본값은 __kwdefaults__ 속성에 들어 있다. 
- 그러나 인수명은 __code__ 속성에 들어 있는데, 이 속성은 여러 속성을 담고 있는 code 객체를 가리킨다. 

In [29]:
def clip(text, max_len=80):
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다."""
    
    end=None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        # 0부터 max_len 사이에서 역순으로 ' '를 찾음
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            # max_len부터 ' ' 를 찾음
            if space_after >= 0:
                end = space_after
    
    if end is None: # 공백이 없다.
        end = len(text)
    return text[:end].rstrip()

In [30]:
clip.__defaults__ # default values

(80,)

In [31]:
clip.__code__

<code object clip at 0x109a1a3a0, file "/var/folders/cx/bz8vssz11fn_6rj2n8nr1p3w0000gn/T/ipykernel_54031/3315024375.py", line 1>

In [32]:
clip.__code__.co_varnames # 코드 내부 변수

('text', 'max_len', 'end', 'space_before', 'space_after')

In [33]:
clip.__code__.co_argcount # arg 개수

2

In [35]:
from inspect import signature
sig = signature(clip)
sig, str(sig)

(<Signature (text, max_len=80)>, '(text, max_len=80)')

In [36]:
for name, param in sig.parameters.items():
    print(param.kind, ":", name, "=", param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [37]:
sig = signature(tag)
my_tag

{'name': 'img',
 'title': 'Sunset Boulevard',
 'src': 'sunset.jpg',
 'cls': 'framed'}

In [38]:
# tag() 에서 가져온 함수 시그니처를 arg dict 에 바인딩
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [39]:
del my_tag['name']
bound_args = sig.bind(**my_tag)
bound_args # 필수인 name 이 없어 error 발생

TypeError: missing a required argument: 'name'

# 5.9 함수 애너테이션
- 매개변수 반환값에 메타데이터를 추가할 수 있는 구문을 제공

In [40]:
def clip2(text:str, max_len:'int>0'=80) -> str: # 함수 선언에 애너테이션 추가
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다."""
    end=None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        # 0부터 max_len 사이에서 역순으로 ' '를 찾음
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            # max_len부터 ' ' 를 찾음
            if space_after >= 0:
                end = space_after
    
    if end is None: # 공백이 없다.
        end = len(text)
    return text[:end].rstrip()

In [41]:
clip2('Hello world this is good',-100) # 애너테이션은 검사, 검증 등은 하지 않음

'Hello world this is'

In [42]:
clip2.__annotations__

{'text': str, 'max_len': 'int>0', 'return': str}

In [43]:
sig2 = signature(clip2)
sig2.return_annotation

str

In [44]:
for param in sig2.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int>0'       : max_len = 80


# 5.10 함수형 프로그래밍을 위한 패키지
- operator와 functool 같은 패키지를 쓰면 함수형 코딩 가능

In [45]:
from functools import reduce

def fact(n):
    return reduce(lambda a,b:a*b, range(1, n+1))

fact(10)

3628800

In [47]:
from operator import mul
# operator 모듈은 수십 개의 연산자에 대응하는 함수를 제공

def fact2(n):
    return reduce(mul, range(1,n+1))

fact2(10)

3628800

In [48]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [50]:
for city in sorted(metro_data, key=itemgetter(0,1)):
    print(city)

('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))
('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))


In [52]:
from collections import namedtuple

# itemgetter()는 시퀀스 뿐 아니라 매핑 및 __getitem__()을 구현한 모든 클래스를 지원한다.
# itemgetter()의 형제인 attrgetter()는 이름으로 객체 속성을 추출하는 함수를 생성한다.

LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
              for name, cc, pop, (lat, long) in metro_data]
metro_areas[0]


Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [53]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
name_lat(metro_areas[0])

('Tokyo', 35.689722)

In [54]:
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [55]:
metro_areas2 = [Metropolis(name, cc, pop, LatLong(*loc))
              for name, cc, pop, loc in metro_data]
metro_areas2[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [58]:
import operator
[name for name in dir(operator) if not name.startswith('_')][:20]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod']

In [59]:
from operator import methodcaller
#methodcaller 는 인수로 전달받은 객체의 메서드를 호출한다.

s = 'The time has come'
upcase = methodcaller('upper')

upcase(s)

'THE TIME HAS COME'

In [60]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

## 5.10.2 functools.partial() 로 인수 고정
- 함수를 부분적으로 실행할 수 있도록 해주는 고위함수
- 함수에 적용시 일부 인수를 고정한 callable 생성

In [61]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
triple(7)

21

In [62]:
list(map(triple, range(1,10))) #3의 배수 출력 

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [63]:
# partial()을 이용해서 편리한 유니코드 정규화 함수 만들기
import unicodedata
nfc = partial(unicodedata.normalize, 'NFC')

s1 = 'café'
s2 = 'cafe\u0301'

s1 == s2, nfc(s1) == nfc(s2)

(False, True)

In [65]:
# tag() 함수에 적용한 partial() 함수
tag
picture = partial(tag, 'img', cls='pic-frame')
picture(src='wumpus.jpg')

'<img class="pic-frame" src="wumpus.jpg" />'

In [66]:
tag, picture, picture.func, picture.args, picture.keywords

(<function __main__.tag(name, *content, cls=None, **attrs)>,
 functools.partial(<function tag at 0x103e0e040>, 'img', cls='pic-frame'),
 <function __main__.tag(name, *content, cls=None, **attrs)>,
 ('img',),
 {'cls': 'pic-frame'})